In [1]:
import requests
import json
import pandas as pd
import numpy as np
import os

In [2]:
## Get Geolocations for federal states and districts
with open('C:/Users/simon/Jupyter/Covid/Dashboard/landkreise_simplify200.geojson', 'r') as read_file:
    geo_kreis = json.load(read_file)

with open('C:/Users/simon/Jupyter/Covid/Dashboard/bundeslaender_simplify200.geojson', 'r') as read_file:
    geo_land = json.load(read_file)

In [3]:
## Get population stats for federal states and districts
df_einwohner_kreis = pd.read_csv('C:/Users/simon/Jupyter/Covid/Dashboard/Einwohner_kreis.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})

df_einwohner_land = pd.read_csv('C:/Users/simon/Jupyter/Covid//Dashboard/Einwohner_land.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})
df_einwohner_land.head()

,Bundesland,Einwohner
0,Baden-Württemberg,11100394
1,Bayern,13124737
2,Berlin,3669491
3,Brandenburg,2521893
4,Bremen,681202


In [4]:
## Get Covid stats from RKI
covid_url = 'https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.geojson'
r = requests.get(covid_url)
r.status_code

200

In [5]:
a = json.loads(r.content)
df = pd.json_normalize(a['features'])

df.head()

,type,geometry,properties.ObjectId,properties.IdBundesland,properties.Bundesland,properties.Landkreis,properties.Altersgruppe,properties.Geschlecht,properties.AnzahlFall,properties.AnzahlTodesfall,properties.Meldedatum,properties.IdLandkreis,properties.Datenstand,properties.NeuerFall,properties.NeuerTodesfall,properties.Refdatum,properties.NeuGenesen,properties.AnzahlGenesen,properties.IstErkrankungsbeginn,properties.Altersgruppe2
0,Feature,None,1,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-09-30T00:00:00Z,01001,"05.09.2021, 00:00 Uhr",0,-9,2020-09-30T00:00:00Z,0,1,0,Nicht übermittelt
1,Feature,None,2,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-10-29T00:00:00Z,01001,"05.09.2021, 00:00 Uhr",0,-9,2020-10-29T00:00:00Z,0,1,0,Nicht übermittelt
2,Feature,None,3,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-03T00:00:00Z,01001,"05.09.2021, 00:00 Uhr",0,-9,2020-11-03T00:00:00Z,0,1,0,Nicht übermittelt
3,Feature,None,4,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-20T00:00:00Z,01001,"05.09.2021, 00:00 Uhr",0,-9,2020-11-19T00:00:00Z,0,1,1,Nicht übermittelt
4,Feature,None,5,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-23T00:00:00Z,01001,"05.09.2021, 00:00 Uhr",0,-9,2020-11-18T00:00:00Z,0,1,1,Nicht übermittelt


In [6]:
## Clean-up and data preparation
df.columns = df.columns.str.replace('properties.', '')
df = df[['IdBundesland', 'Bundesland', 'Landkreis', 'IdLandkreis', 'AnzahlFall', 'Altersgruppe', 'AnzahlTodesfall', 'Geschlecht', 'Meldedatum']]
df['Meldedatum'] = df['Meldedatum'].str.replace('T00:00:00Z', '')
df['Meldedatum'] = pd.to_datetime(df['Meldedatum'])

df = df.merge(df_einwohner_kreis, how='left', left_on='IdLandkreis', right_on='RS')
df = df.merge(df_einwohner_land, how='left', left_on='Bundesland', right_on='Bundesland') 

df_gender = pd.get_dummies(df['Geschlecht']).mul(df['AnzahlFall'],0)
df_gender.columns = ['Sex_M', 'Sex_F', 'Sex_Unbekannt']
df = df.join(df_gender)

df_age = pd.get_dummies(df['Altersgruppe']).mul(df['AnzahlFall'],0)
df = df.join(df_age)



df.drop(['RS', 'Kreis'], axis=1, inplace=True)

df.head()

<ipython-input-6-157b8c82909c>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('properties.', '')


,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
0,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-09-30,90164.0,...,1,0,0,1,0,0,0,0,0,0
1,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-10-29,90164.0,...,1,0,0,1,0,0,0,0,0,0
2,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-03,90164.0,...,1,0,0,1,0,0,0,0,0,0
3,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-20,90164.0,...,1,0,0,1,0,0,0,0,0,0
4,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-23,90164.0,...,1,0,0,1,0,0,0,0,0,0


In [7]:
df_bund = df.groupby(by=['Meldedatum']).agg({
    'AnzahlFall':'sum',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})


#df_bund.set_index('Region', append=True, inplace=True)
#df_bund = df_bund.swaplevel('Region', 'Meldedatum')

df_bund['Fall_Last7'] = df_bund['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_bund['Fall_MA7'] = df_bund['AnzahlFall'].rolling(7).mean()
df_bund['R7'] = (df_bund['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_bund['Einwohner'] = df_einwohner_land['Einwohner'].sum()
df_bund['Fall_100K'] = ((df_bund['Fall_Last7'] / df_bund['Einwohner']) * 100000).round(2)

df_bund.reset_index(inplace=True)
df_bund['Region'] = 'Deutschland'
df_bund['Typ'] = 'Nation'


df_bund.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt,Fall_Last7,Fall_MA7,R7,Einwohner,Fall_100K,Region,Typ
568,2021-08-31,14363,6,7200,6934,229,604,2893,5266,4565,738,294,3,68018.0,9716.857143,1.06,83166711,81.79,Deutschland,Nation
569,2021-09-01,14301,4,7040,6953,308,685,2971,5263,4368,743,263,8,69607.0,9943.857143,1.06,83166711,83.70,Deutschland,Nation
570,2021-09-02,12092,2,5892,5975,225,518,2639,4495,3510,675,241,14,70122.0,10017.428571,1.06,83166711,84.31,Deutschland,Nation
571,2021-09-03,10756,0,5290,5223,243,511,2323,3863,3187,626,208,38,70531.0,10075.857143,1.06,83166711,84.81,Deutschland,Nation
572,2021-09-04,6830,1,3336,3350,144,368,1465,2443,2004,410,131,9,69042.0,9863.142857,1.02,83166711,83.02,Deutschland,Nation


In [8]:
df_land = df.groupby(by=['Bundesland', 'Meldedatum']).agg({
    'IdBundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_y': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})

df_land.rename(columns={'Einwohner_y': 'Einwohner'}, inplace=True)

#df_land.index.names = ['Region', 'Meldedatum']

df_land['Fall_MA4'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_land['Fall_MA7'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_land['R4'] = df_land['Fall_MA4'].pct_change(periods=4) + 1
df_land['R7'] = (df_land['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_land['Fall_Last7'] = df_land['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_land['Fall_100K'] = ((df_land['Fall_Last7'] / df_land['Einwohner']) * 100000).round(2)

#

df_land.reset_index(inplace=True)
df_land['Region'] = df_land['Bundesland']
df_land['Typ'] = 'Bundesland'

df_land.tail()

,Bundesland,Meldedatum,IdBundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
8705,Thüringen,2021-08-31,16,126,2133378,0,66,59,1,10,...,4,0,64.25,77.428571,0.650633,1.13,542.0,25.41,Thüringen,Bundesland
8706,Thüringen,2021-09-01,16,136,2133378,0,68,67,1,8,...,6,1,86.50,83.285714,1.042169,1.18,583.0,27.33,Thüringen,Bundesland
8707,Thüringen,2021-09-02,16,102,2133378,0,52,47,3,5,...,4,0,110.50,85.000000,1.818930,1.22,595.0,27.89,Thüringen,Bundesland
8708,Thüringen,2021-09-03,16,138,2133378,0,64,72,2,7,...,6,0,125.50,90.428571,2.173160,1.20,633.0,29.67,Thüringen,Bundesland
8709,Thüringen,2021-09-04,16,59,2133378,0,30,29,0,7,...,2,0,108.75,92.142857,1.692607,1.19,645.0,30.23,Thüringen,Bundesland


In [9]:
df_kreis = df.groupby(by=['Landkreis', 'Meldedatum']).agg({
    'IdLandkreis': 'first',
    'Bundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_x': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})
    
df_kreis.rename(columns={'Einwohner_x': 'Einwohner'}, inplace=True)


#df_kreis.index.names = ['Region', 'Meldedatum']

df_kreis['Fall_MA4'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_kreis['Fall_MA7'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_kreis['R4'] = df_kreis['Fall_MA4'].pct_change(periods=4) + 1
df_kreis['R7'] = (df_kreis['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_kreis['Fall_Last7'] = df_kreis['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_kreis['Fall_100K'] = ((df_kreis['Fall_Last7'] / df_kreis['Einwohner']) * 100000).round(2)

df_kreis.reset_index(inplace=True)
df_kreis['Region'] = df_kreis['Landkreis']
df_kreis['Typ'] = 'Landkreis'

df_kreis.tail()

,Landkreis,Meldedatum,IdLandkreis,Bundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
170015,Städteregion Aachen,2021-08-31,05334,Nordrhein-Westfalen,100,557026.0,0,58,42,0,...,3,0,50.75,74.714286,0.440347,0.82,523.0,93.89,Städteregion Aachen,Landkreis
170016,Städteregion Aachen,2021-09-01,05334,Nordrhein-Westfalen,98,557026.0,0,42,56,0,...,1,0,60.75,75.142857,0.642857,0.85,526.0,94.43,Städteregion Aachen,Landkreis
170017,Städteregion Aachen,2021-09-02,05334,Nordrhein-Westfalen,109,557026.0,0,56,51,2,...,0,0,84.50,76.142857,1.138047,0.89,533.0,95.69,Städteregion Aachen,Landkreis
170018,Städteregion Aachen,2021-09-03,05334,Nordrhein-Westfalen,77,557026.0,0,42,34,1,...,2,0,96.00,69.571429,1.699115,0.86,487.0,87.43,Städteregion Aachen,Landkreis
170019,Städteregion Aachen,2021-09-04,05334,Nordrhein-Westfalen,25,557026.0,0,9,16,0,...,0,0,77.25,64.857143,1.522167,0.87,454.0,81.50,Städteregion Aachen,Landkreis


In [10]:
## Final dataset
df_merged = pd.concat([df_bund, df_land, df_kreis])

## The geolocations for the districts of Berlin are not included in the dataset, 
## to have Berlin displayed on the choropleth map, I substitute the values for the federal state of Berlin.
df_merged.loc[df_merged['Bundesland'] == 'Berlin', 'IdLandkreis'] = '11000'

df_merged.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,...,Einwohner,Fall_100K,Region,Typ,Bundesland,IdBundesland,Fall_MA4,R4,Landkreis,IdLandkreis
170015,2021-08-31,100,0,58,42,0,4,32,31,24,...,557026.0,93.89,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,50.75,0.440347,Städteregion Aachen,05334
170016,2021-09-01,98,0,42,56,0,1,24,33,34,...,557026.0,94.43,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,60.75,0.642857,Städteregion Aachen,05334
170017,2021-09-02,109,0,56,51,2,4,33,27,42,...,557026.0,95.69,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,84.50,1.138047,Städteregion Aachen,05334
170018,2021-09-03,77,0,42,34,1,6,16,24,24,...,557026.0,87.43,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,96.00,1.699115,Städteregion Aachen,05334
170019,2021-09-04,25,0,9,16,0,0,3,9,8,...,557026.0,81.50,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,77.25,1.522167,Städteregion Aachen,05334


In [11]:
if os.path.exists('covid_panel_backup.csv'):
    os.remove('covid_panel_backup.csv')

In [12]:
os.rename('covid_panel.csv', 'covid_panel_backup.csv')

In [13]:
df_merged.to_csv('covid_panel.csv')